In [1]:
from transformers import AutoTokenizer
from model import HateXplainModel
from dataset import HateXplainDataModule
import pytorch_lightning as pl
import json
from argparse import Namespace

/home/phuctth/miniconda3/envs/limit-bert/lib/python3.8/site-packages/pytorch_lightning/metrics/__init__.py:43: LightningDeprecationWarning: `pytorch_lightning.metrics.*` module has been renamed to `torchmetrics.*` and split off to its own package (https://github.com/PyTorchLightning/metrics) since v1.3 and will be removed in v1.5
  rank_zero_deprecation(


In [2]:
label_map = {0:"Hate speech", 1: "Normal", 2: "Offensive"}
config = Namespace()
vars(config).update(json.load(open("checkpoints/config.json","r")))
tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased")
model = HateXplainModel.load_from_checkpoint("checkpoints/epoch=3-step=3847.ckpt", config=config, tokenizer=tokenizer)

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
model.eval();

In [4]:
label, rationale = model("Why are you repeating yourself are you a little retarded?")
print(label_map[label])
print(rationale)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0])
Offensive
['little retarded?']


In [5]:
trainer=pl.Trainer()
hatexplain_dm = HateXplainDataModule(config, tokenizer)
trainer.test(model, datamodule=hatexplain_dm)

GPU available: True, used: False
TPU available: False, using: 0 TPU cores
/home/phuctth/miniconda3/envs/limit-bert/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:596: UserWarning: GPU available but not used. Set the gpus flag in your trainer `Trainer(gpus=1)` or script `--gpus=1`.
  rank_zero_warn(
Reusing dataset hatexplain (/home/phuctth/.cache/huggingface/datasets/hatexplain/plain_text/1.0.0/df474d8d8667d89ef30649bf66e9c856ad8305bef4bc147e8e31cbdf1b8e0249)
/home/phuctth/miniconda3/envs/limit-bert/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:102: UserWarning: The dataloader, test dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Cls F1: 0.688, Rationale F1: 0.872

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{}
--------------------------------------------------------------------------------


[{}]